# DistilBERT 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install sentencepiece
! pip install transformers
! pip install tensorflow-addons
! pip install tf-models-official
! pip install --upgrade tensorflow-hub

     |████████████████████████████████| 1.2MB 6.3MB/s 
     |████████████████████████████████| 2.2MB 6.9MB/s 
     |████████████████████████████████| 3.3MB 23.4MB/s 
     |████████████████████████████████| 870kB 39.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=8848e73b999455ddb46e2d38b35391d81d510752e9cd82180bbaf46a08501222
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 706kB 5.4MB/s 
     |████████████████████████████████| 1.1MB 6.2MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 645kB 14.8MB/s 
     |████████████████████████████████| 37.6MB 124kB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 358kB 40.3MB/s 
     |████████████████████████████████| 174kB 40.1MB/s 
  Created wheel for py-cpuinfo: filen

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers


# DistilBERT ENCODER Helper

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    
    all_tokens = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
    
    return np.array(all_tokens)

#Building DistilBERT Model Helper

In [ ]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Uploading Data

In [ ]:
train_data = pd.read_csv('/content/gdrive/MyDrive/all_data/train_data.csv')
train_data.head()

test_data = pd.read_csv('/content/gdrive/MyDrive/all_data/test_data.csv')
train_data.head()

,Unnamed: 0,tweet_id,tweets,bot
5000,5000,b3bca488109c80a6f37419fb4933558c,"I learned law so well, the day I graduated I s...",bot
5001,5001,b3bca488109c80a6f37419fb4933558c,Great Review of The Ruins of Gorlan by John Fl...,bot
5002,5002,b3bca488109c80a6f37419fb4933558c,"When Life Gives You Questions, Google has Answ...",bot
5003,5003,b3bca488109c80a6f37419fb4933558c,If you're listening to a rock star in order to...,bot
5004,5004,b3bca488109c80a6f37419fb4933558c,Hilarious What Went Wrong T-shirt - Love It! ...,bot


# Train Model

In [ ]:
transformer_layer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
model = build_model(transformer_layer, max_len=160)
model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

# Converting Labels to Binary

In [ ]:
batch_1 = train_data[train_data.columns[-2:]]

batch_1.bot[batch_1.bot == 'bot'] = 1
batch_1.bot[batch_1.bot == 'human'] = 0

batch_1.head()


batch_2 = test_data[test_data.columns[-2:]]


batch_2.bot[batch_2.bot == 'bot'] = 1
batch_2.bot[batch_2.bot == 'human'] = 0

batch_2.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1021: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


,tweets,bot
5000,Yellows building. HRRR did a good job with thi...,0
5001,RT @AWxNYC: #SNOW starting to stick to the roa...,0
5002,@mattlanza I’ll prob sleep thru most of it hah,0
5003,@KevinMyattWx Good ole weather,0
5004,RT @_CTWeather: @MikeTFox5 @dougkammerer @hbwx...,0


# Model

In [ ]:
train_input = bert_encode(batch_1.tweets.values, tokenizer, max_len=160)
test_input = bert_encode(batch_2.tweets.values, tokenizer, max_len=160)
train_labels = batch_1.bot.values

In [ ]:
train_history = model.fit(train_input.astype(np.float32), train_labels.astype(np.float32), epochs=1)


test_pred = model.predict(test_input.astype(np.float32), verbose=1)

8094/8094 [==============================] - 829s 102ms/step


# Accuracy Score

In [ ]:
target = test_pred.round().astype(int)

from sklearn.metrics import accuracy_score

original = []
final = []

for i in target:
  for j in i:
    final.append(j)

for i in batch_2.bot.values:
  original.append(i)


accuracy_score(original, final)